### Query MySQL for data

In [1]:
import pandas as pd
import network_vis_helper as visnet
import api_wrapper as api
pd.set_option('display.max_colwidth', None)

## reading in data until the database is established
activity = pd.read_csv('/Users/dankoban/Documents/EM6586_DB_Management/final_project_twitter_data/activity.csv')
activity = activity[['author_id', 'author_screen_name', 'id', 'conversation_id', 'text', 'created_at', 'lang', 
                     'in_reply_to_user_id', 'geo', 'retweet_count', 'reply_count', 'like_count', 'quote_count',
                     'referenced_tweet_type', 'referenced_tweet_id']]
print(len(activity))
activity.drop_duplicates('id', inplace = True)
print(len(activity))
activity.head()

/Users/dankoban/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


256661
234163


,author_id,author_screen_name,id,conversation_id,text,created_at,lang,in_reply_to_user_id,geo,retweet_count,reply_count,like_count,quote_count,referenced_tweet_type,referenced_tweet_id
0,4322065997,thereal_truther,1370489381841104898,1370447737838317574,@RussellBoullion @emmahn6 @naomirwolf @HenningTveit @andrewcuomo This is 2017. \n\nThe rates had been rising and hit a record high that year.\n\nThis year the trend will cease. https://t.co/WdhDDQruD4,2021-03-12T21:38:25.000Z,en,1.362841e+18,NaN,0,0,0,0,replied_to,1.370488e+18
1,4322065997,thereal_truther,1370488997105913858,1370447737838317574,@RussellBoullion @emmahn6 @naomirwolf @HenningTveit @andrewcuomo No rise.\n\nhttps://t.co/mONB4kOt2N,2021-03-12T21:36:54.000Z,en,1.362841e+18,NaN,0,0,0,0,quoted,1.367243e+18
2,4322065997,thereal_truther,1370488898292387840,1370447737838317574,"@RussellBoullion @emmahn6 @naomirwolf @HenningTveit @andrewcuomo Yes, I've been tracking this.\n\nSuicides are DOWN in strict lockdown countries like New Zealand, Canada, USA &amp; Peru.\n\nStable everywhere else like Australia &amp; the UK.\n\nChild mortality dropped DRAMATICALLY. https://t.co/C1NcX105Kg",2021-03-12T21:36:30.000Z,en,1.362841e+18,NaN,0,1,0,0,replied_to,1.370487e+18
3,4322065997,thereal_truther,1370485453107777539,1370447737838317574,@RussellBoullion @emmahn6 @naomirwolf @HenningTveit @andrewcuomo What suicides?\n\nThey are down this year.\n\nChild mortality dropped dramatically during the lockdowns.,2021-03-12T21:22:49.000Z,en,1.362841e+18,NaN,0,5,0,0,replied_to,1.370467e+18
4,4322065997,thereal_truther,1370484201955356672,1370419244580622340,"@Science78560422 @stacey_rudin @aid_taz @dockaurG To her those are all ""lockdown deaths.""",2021-03-12T21:17:50.000Z,en,1.352630e+18,NaN,0,0,0,0,replied_to,1.370478e+18


In [2]:
activity['referenced_tweet_type'].value_counts()

original_tweet    87798
replied_to        74712
retweeted         59427
quoted            12226
Name: referenced_tweet_type, dtype: int64

In [3]:
el = pd.read_csv('/Users/dankoban/Documents/EM6586_DB_Management/final_project_twitter_data/el.csv')
print(len(el))
len(el['author_screen_name_from'].unique().tolist())
el.head()

646166


,author_id_from,author_screen_name_from,status_id,to,edge_type
0,4322065997,thereal_truther,1370464773138120705,antivax,hashtag
1,4322065997,thereal_truther,1369703378633121792,covid,hashtag
2,4322065997,thereal_truther,1369703378633121792,antivax,hashtag
3,404290687,crwriter1,1370479219508183047,investigation,hashtag
4,404290687,crwriter1,1370475694258597892,writing,hashtag


In [4]:
mentioned_users = el['to'][el['edge_type'] == 'mention'].unique().tolist()
print('mentioned user count: ' + str(len(mentioned_users)))

mentioned user count: 60405


### Visualize networks

In [5]:
weighted_el = visnet.get_weighted_el(el)
visnet.render_graph(weighted_el, edge_type = 'hashtag', edge_weight_threshold = 20)

In [6]:
top_hashtags = (weighted_el[weighted_el['edge_type'] == 'hashtag']
                .groupby('target').sum().reset_index()
                .sort_values('weight', ascending = False))
top_hashtags.head(30)

,target,weight
1648,COVID19,5399
18740,vaccines,1047
18691,vaccine,1022
11456,VaccinesWork,897
1646,COVID,652
13544,coronavirus,555
14977,health,532
2467,Covid19,381
2510,CovidVaccine,361
16932,pharma,351


In [7]:
activity['conversation_id'].value_counts()

851513700225888257     2134
922168727457513472     1726
1369795885308256261     193
1135250346383745025     190
1293319380818898950     142
                       ... 
1217106649300770817       1
1367761104676519936       1
1368248761588318209       1
1131392482460262400       1
1370272965800103937       1
Name: conversation_id, Length: 194073, dtype: int64

In [8]:
activity[['author_screen_name', 'id', 'created_at', 'text', 'like_count']].sort_values('like_count', ascending = False).head(10)

,author_screen_name,id,created_at,text,like_count
209116,UNICEF,1367618399560466440,2021-03-04T23:30:10.000Z,"💜💙\n\n@BTS_twt's messages of solidarity have given hope to children and young people around the world during COVID-19.\n\nToday, we are proud to announce that we are renewing our joint commitment to creating a kinder, safer and more connected world. Are you with us?\n\n#BTSLoveMyself",172265
209115,UNICEF,1367618426475327488,2021-03-04T23:30:16.000Z,"The first step to creating a kinder world? Learning to love yourself.\n \nAs we face the second year of the COVID-19 pandemic, @bts_twt reflect on the importance of connection and self-acceptance.\n \n#BTSLoveMyself",118748
209094,UNICEF,1367940858159505415,2021-03-05T20:51:30.000Z,"⚡️ A look back on the #BTSLoveMyself campaign, as @BTS_twt renew their commitment to helping UNICEF #ENDviolence and promote self-esteem and well-being.\n\nhttps://t.co/Z5BIGJ12j1",38483
252763,Surgeon_General,1351887300687503360,2021-01-20T13:40:23.000Z,"I've been asked by the Biden team to step down as Surgeon General. Its been the honor of my life to serve this Nation, and I will do all I can to ensure everyone has an equal opportunity to achieve and maintain health. See my full statement at: https://t.co/pCkbpealt8",16558
35649,forrestmaready,1278089502859366402,2020-06-30T22:14:17.000Z,It's just a mask?\n\nIt's just two weeks.\nIt's just non-essential businesses.\nIt's just to keep from overwhelming the hospitals.\nIt's just until cases go down more.\nIt's just to keep others from being scared.\nIt's just for a few more weeks.,15369
78465,PeterHotez,1370184139396280324,2021-03-12T01:25:30.000Z,"Looking for things to criticize or question in the @JoeBiden speech tonight. For me, nada. A home run. Thank you. \n\nTurns out we elected a really good POTUS.",13758
83712,aprillynnclark,1365320548935487492,2021-02-26T15:19:19.000Z,https://t.co/bY60vqhxVI,11918
171487,CDCDirector,1351991282244214787,2021-01-20T20:33:35.000Z,"It is a privilege to join @CDCgov. For 75 years, CDC has carried out a mission to protect America’s safety, health, &amp; security at home &amp; abroad. I recognize the seriousness of the moment - the toll that #COVID19 has had on America is truly heartbreaking. https://t.co/NLVqme863t",11386
55653,UNICEFUSA,1367911550162513922,2021-03-05T18:55:02.000Z,"We love to see it!\n\nDuring the pandemic, @BTS_twt's messages of solidarity have given hope to children and young people everywhere. In their continued commitment to UNICEF, they're working to make the world a kinder place. #BTSLoveMyself 👉 https://t.co/So7mhALaNN https://t.co/roDNxfCkja",10728
171482,CDCDirector,1352285084720754690,2021-01-21T16:01:03.000Z,"Although I never anticipated leading the response to #COVID19, when the nation is coding, if you are called to serve, you serve. You run to take care of people, to give them hope and help them have a fighting chance to come back stronger.",10042
